In [1]:
import mne
import numpy as np
from mne.preprocessing import ICA

In [2]:
import pandas as pd
subject_pd=['PD1001','PD1021','PD1031','PD1061','PD1091','PD1101','PD1151'
                    ,'PD1201','PD1251','PD1261','PD1311','PD1571','PD1661','PD1681']
subject_hc=['Control1021','Control1041','Control1061','Control1081','Control1101'
                ,'Control1111','Control1191','Control1201','Control1211','Control1231','Control1291'
                ,'Control1351','Control1381','Control1411']


In [ ]:
subject_files_pd=[]
subject_files_hc=[]

for subject in subject_pd:
    file_path = f"IowaDataset/Raw data/{subject}.vhdr"
    subject_files_pd.append(file_path)

for subject in subject_hc:
    file_path = f"IowaDataset/Raw data/{subject}.vhdr"
    subject_files_hc.append(file_path)

print(subject_files_pd)
print(subject_files_hc)


In [ ]:
print(len(subject_files_pd))
print(len(subject_files_hc))

In [5]:
def set_montage(raw_data):
    montage = mne.channels.make_standard_montage('biosemi32')
    raw_data.set_montage(montage, on_missing='warn')
    return raw_data

In [6]:
def bandpass_filter(raw_data, l_freq=0.5, h_freq=50.0):
    raw_data.filter(l_freq=l_freq, h_freq=h_freq)
    return raw_data

In [7]:
def find_ecg_via_temporal_channels(ica, raw_data):
    # Experimentally identify ECG-like artifacts using temporal channels
    ecg_indices, ecg_scores = ica.find_bads_ecg(raw_data, ch_name='T7')
    ica.exclude += ecg_indices  # Exclude identified ECG-like components
    return ica

In [8]:
def segment_data(raw_data, duration=1.0):
    events = mne.make_fixed_length_events(raw_data, duration=duration)
    epochs = mne.Epochs(raw_data, events, tmin=0, tmax=duration, baseline=None, preload=True)
    eeg_data = epochs.get_data()  # Shape should be (180, 32, 512) if 3 mins, 32 channels, 512 samples/s
    return eeg_data

In [9]:
def apply_ica(raw_data, n_components=29):
    ica = ICA(n_components=n_components, random_state=97, max_iter="auto")
    ica.fit(raw_data)
    
    # Detect artifacts
    eog_indices, _ = ica.find_bads_eog(raw_data,ch_name=['Fp2', 'F8'],threshold=1.96)  # Detect eye blink components
    
    # Mark components for removal
    ica.exclude = eog_indices
    # Experimental ECG detection
    ica = find_ecg_via_temporal_channels(ica, raw_data)
    
    # Apply ICA to remove artifacts
    raw_data = ica.apply(raw_data)
    return raw_data

In [10]:
# Define frequency bands
freq_bands = {
    "delta": (1, 4),
    "theta": (4, 8),
    "alpha": (8, 12),
    "beta": (13, 30),
    "gamma": (30, 48)
}

In [11]:
def compute_psd(eeg_data, sfreq):
    psd_features = {}
    for band, (low, high) in freq_bands.items():
        psd_band, _ = mne.time_frequency.psd_array_multitaper(
            eeg_data, sfreq=sfreq, fmin=low, fmax=high, adaptive=True, normalization='full'
        )
        psd_features[band] = psd_band.mean(axis=2)  # Average PSD across time
    return psd_features

In [12]:
# Define a function to process a single subject and extract PSD features
def process_subject(raw_data, sfreq):
    # Apply bandpass filtering and artifact removal here as per previous preprocessing steps
    set_montage(raw_data)
    raw_filtered = bandpass_filter(raw_data)  # Assuming bandpass_filter function is defined
    raw_filtered = apply_ica(raw_filtered)
    epochs = segment_data(raw_filtered)  # Assuming segment_data function is defined to get (180, 32, 512)
    eeg_data = epochs[:,:29, :500]  # Shape (180, 29, 500)

    # Compute PSD features for this subject
    psd_features = compute_psd(eeg_data, sfreq)  # Dictionary with PSD for each frequency band
    return psd_features

In [13]:
def collect_psd_features(subject_files, sfreq):
    all_psd_features = {'delta': [], 'theta': [], 'alpha': [], 'beta': [], 'gamma': []}
    for subject_file in subject_files:
        # Load subject's data
        raw_data = mne.io.read_raw_brainvision(subject_file, preload=True)
        raw_data.crop(tmax=120.)

        channels_to_keep = ['Fp1', 'AF3', 'F7', 'F3', 'FC1', 'FC5', 'T7', 'C3', 
                            'CP1', 'CP5', 'P7', 'P3', 'O1', 'Oz', 'O2', 
                            'P4', 'P8', 'CP6', 'CP2', 'C4', 'T8', 'FC6', 'FC2', 
                            'F4', 'F8', 'AF4', 'Fp2', 'Fz', 'Cz']
        raw_data = raw_data.pick_channels(channels_to_keep)
        # 29 channles
        
        # Process each subject's data to extract PSD features
        psd_features = process_subject(raw_data, sfreq)
        print("Shape of psd_features[delta] in each sub: ",np.asarray(psd_features['delta']).shape)
        # Append features for each frequency band
        for band in all_psd_features.keys():
            all_psd_features[band].append(psd_features[band])  # Each entry: (180, 32)
    return all_psd_features

In [ ]:
sfreq = 500  # Sample frequency as given

# psd_features_pd = collect_psd_features(subject_files_pd_on, sfreq)
psd_features_pd = collect_psd_features(subject_files_pd, sfreq)

In [ ]:
np.asarray(psd_features_pd['beta']).shape

In [ ]:
psd_features_hc = collect_psd_features(subject_files_hc, sfreq)

In [ ]:
np.asarray(psd_features_hc['delta']).shape

In [21]:
# Combine PSD features into a dataset for SVM classification
psd_features = {'delta': psd_features_pd['delta'] + psd_features_hc['delta'],
                'theta': psd_features_pd['theta'] + psd_features_hc['theta'],
                'alpha': psd_features_pd['alpha'] + psd_features_hc['alpha'],
                'beta': psd_features_pd['beta'] + psd_features_hc['beta'],
                'gamma': psd_features_pd['gamma'] + psd_features_hc['gamma']}

In [ ]:
np.asarray(psd_features['delta']).shape

In [39]:
# Prepare dataset (example: delta band, label 0 for HC and 1 for PD_ON,OFF)
def prepare_dataset(psd_features, num_pd=14, num_hc=14, label_pd=1, label_hc=0):
    # Combine PSD features for PD_ON and HC
    data = []
    labels = []
    
    # Append PD subjects
    for i in range(num_pd):
        psd_pd_on_off = psd_features['gamma'][i]  # Shape (180, 32)
        data.append(psd_pd_on_off)
        labels.extend([label_pd] * psd_pd_on_off.shape[0])  # Label each sample in this subject's data as PD_ON
    
    # Append HC subjects
    for i in range(num_hc):
        psd_hc = psd_features['gamma'][i + num_pd]  # Assuming next in sequence
        data.append(psd_hc)
        labels.extend([label_hc] * psd_hc.shape[0])  # Label each sample in this subject's data as HC
    
    # Stack data into final dataset
    data = np.vstack(data)  # Resulting shape: (5580, 32)
    labels = np.array(labels)  # Resulting shape: (5580,)
    return data, labels

In [40]:
# Prepare dataset (using only δ band for this example)
data, labels = prepare_dataset({'gamma': psd_features['gamma']})

In [ ]:
# Verify the data shape
print("Full dataset shape:", data.shape)  # Should be (5580, 32)
print("Label distribution:", np.unique(labels, return_counts=True))  # Should show counts for PD_ON and HC

In [42]:
import pickle

# Save using pickle
with open('IowaDataset/processed/gamma.pkl', 'wb') as f:
    pickle.dump(
        {
            'data': data, 
            'labels': labels
        },
    f)